In [1]:
import sys
import os
root_dir = os.getcwd().replace('/Notebooks','')
sys.path.insert(1, root_dir)
sys.path.append('../modules/')
import pandas as pd
import numpy as np
from datetime import datetime as dt
import glob
from zipline.utils.run_algo import load_extensions
from zipline.data import bundles
from zipline.data.data_portal import DataPortal
from zipline.utils.calendar_utils import get_calendar
import os
load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)
from zipline.data.bundles import register, unregister
from zipline.data.bundles.csvdir import csvdir_equities
from zipline.utils.calendar_utils import register_calendar, get_calendar
# from exchange_calendars.exchange_calendar_xdse import XDSExchangeCalendar
from logbook import (NestedSetup, NullHandler, Logger, StreamHandler, StderrHandler, 
                     INFO, WARNING, DEBUG, ERROR)
from zipline.api import (order, 
                         record, 
                         symbol,
                         get_datetime,
                         order_target_percent,
                         order_target_value,
                         order_value,
                         set_benchmark,
                         get_open_orders)
from zipline import run_algorithm
from zipline.utils.calendar_utils import get_calendar
from zipline.api import order_target, record, symbol
from pyfolio.tears import create_full_tear_sheet
from pyfolio.utils import extract_rets_pos_txn_from_zipline
import seaborn as sns
import quantstats as qs
import yfinance as yf
from hurst import compute_Hc, random_walk
import warnings
warnings.filterwarnings('ignore')
import ensemble_trading_strategy as ets
%load_ext autoreload
%autoreload 2

## Signal on positive returns and negative returns

In [2]:
def get_signal(df:pd.DataFrame,periods=2):
    """
    this function receives a dataframe and calculte returns
    if the return is higher than 2 percent then it sends 1 as sell signal 
    and if the return is less than 2 percent then it sends -1 as buy signal
    otherwise it sends 0 as hold signal
    """
    returns = df.close.pct_change(periods=2).mul(100)
    signals = np.where(returns > 2,-1,0)
    signals = np.where(returns < -2, 1, returns)
    return signals[-1]

In [3]:
start = '2018-01-01'
end = '2023-01-01'
capital_base = 10000
calendar = get_calendar('NYSE')
symb = "AAPL"
bundle_data = bundles.load("sp50")
window = 90

def initialize(context):
    context.i = 0
    sids = bundle_data.asset_finder.sids
    context.assets = [symbol(symb)]


def handle_data(context, data):
    # Skip first 90 days to get full windows
    context.i += 1
    if context.i < window+1:
        return
    
    for asset in context.assets:
        df = data.history(asset,['open','high','low','close','volume'],bar_count=window,frequency='1d')
        curr_price = data.current(asset,'price')
        signal = ets.group3_ensemble_model_signals(df)
        # signal = get_signal(df)

        if signal == 1:
            order_target_percent(asset, 0.2)
        elif signal == -1:
            order_target_percent(asset, 0,stop_price=(curr_price * 0.9))
        else:
            pass
    

result = run_algorithm(start=pd.Timestamp(start),
                        end=pd.Timestamp(end),
                        initialize=initialize,
                        handle_data=handle_data,
                        capital_base=capital_base,
                        trading_calendar=calendar,
                        bundle='sp50',
                        data_frequency='daily',
                        )


In [4]:
start = result.index[0]
end = result.index[-1]
benchmark = yf.download('^GSPC', start=start, end=end)['Adj Close'].pct_change()
result.index = pd.to_datetime(result.index).tz_convert(None)
result.index = benchmark.index    
qs.reports.full(result['returns'], benchmark = benchmark, match_dates=True, figsize=(8, 4), df = result)

[*********************100%%**********************]  1 of 1 completed


ValueError: Cannot calculate a linear regression if all x values are identical